In [ ]:
%pip install -r requirements.txt

# Imports

In [17]:
import os
import json

from typing import List
from dotenv import load_dotenv

In [18]:
from pydantic import BaseModel

In [19]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.schema import OutputParserException

In [20]:
from langchain_openai import AzureChatOpenAI

# Load Config

In [21]:
# Load environment variables from .env file
load_dotenv()

# Access environment variables
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

In [22]:
# Set the environment variables
os.environ['AZURE_OPENAI_API_KEY'] = azure_api_key
os.environ['AZURE_OPENAI_ENDPOINT'] = azure_endpoint

# LLM Configuration

In [23]:
MODEL_NAME = "gpt-4o-mini"

In [24]:
# Azure OpenAI - GPT-4o or GPT-4o-mini
llm = AzureChatOpenAI(
    deployment_name=MODEL_NAME,
    model_name="gpt-4o",
    temperature=0,
    api_version="2024-12-01-preview",
)

# Output schema

In [25]:
# Define the schema
class MeetingNotes(BaseModel):
    summary: str
    action_items: List[str]

# Prompt

In [26]:
# The prompt template
PROMPT_TEMPLATE = """
You are a meeting assistant.
1. Summarize the meeting transcript below in exactly two sentences.
2. Then list all action items mentioned, each as a separate bullet beginning with a dash.
Return the result strictly as JSON with keys "summary" and "action_items".
Transcript:
{transcript}
"""

# Main Pipeline Function

In [29]:
def extract_meeting_notes(transcript: str) -> dict:
    # Initialize the parser
    parser = PydanticOutputParser(pydantic_object=MeetingNotes)
    
    # Build the prompt
    prompt = PromptTemplate(
        template=PROMPT_TEMPLATE,
        input_variables=["transcript"],
        output_parser=parser
    )
    
    # Format the final prompt
    final_prompt = prompt.format_prompt(transcript=transcript)
    
    # First attempt
    try:
        output = llm(final_prompt.to_messages())
        return parser.parse(output.content).model_dump_json()
    except OutputParserException:
        # Retry with stricter instruction
        retry_prompt = PromptTemplate(
            template="Please output valid JSON only.\n" + PROMPT_TEMPLATE,
            input_variables=["transcript"],
            output_parser=parser
        ).format_prompt(transcript=transcript)
        
        try:
            output = llm(retry_prompt.to_messages())
            return parser.parse(output.content).model_dump_json()
        except OutputParserException:
            return {
                "error": "Failed to parse valid JSON from LLM output after retry.",
                "last_output": output.content
            }

# Test Cases

In [33]:
transcript = """
Alice: Welcome everyone. Today we need to finalize the Q3 roadmap.
Bob: I’ve emailed the updated feature list—please review byFriday.
Carol: I’ll set up the user‐testing sessions next week.
Dan: Let’s push the new UI mockups to staging on Wednesday.
Alice: Great. Also, can someone compile the stakeholder feedback into a slide deck?
Bob: I can handle the slide deck by Monday.
Alice: Thanks, team. Meeting adjourned.
"""

In [35]:
result = extract_meeting_notes(transcript)
json.dumps(result, indent=4, ensure_ascii=False)

'"{\\"summary\\":\\"The team met to finalize the Q3 roadmap, discussing various tasks and deadlines. Action items were assigned to ensure progress on the updated feature list, user-testing sessions, UI mockups, and stakeholder feedback compilation.\\",\\"action_items\\":[\\"- Review the updated feature list by Friday.\\",\\"- Set up user-testing sessions next week.\\",\\"- Push the new UI mockups to staging on Wednesday.\\",\\"- Compile the stakeholder feedback into a slide deck by Monday.\\"]}"'

In [41]:
import pprint
pprint.pprint(result)

('{"summary":"The team met to finalize the Q3 roadmap, discussing various '
 'tasks and deadlines. Action items were assigned to ensure progress on the '
 'updated feature list, user-testing sessions, UI mockups, and stakeholder '
 'feedback compilation.","action_items":["- Review the updated feature list by '
 'Friday.","- Set up user-testing sessions next week.","- Push the new UI '
 'mockups to staging on Wednesday.","- Compile the stakeholder feedback into a '
 'slide deck by Monday."]}')
